# Table 2.   Factors for Insufficient Pain Management, Multivariate Analysis

In [ ]:
import pandas as pd
import numpy as np
import re
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

In [ ]:
data_path = '/Users/jk1/Library/CloudStorage/OneDrive-unige.ch/icu_research/prehospital/analgesia/data/trauma_categories_Rega Pain Study15.09.2025_v2.xlsx'
medic_data_path = '/Users/jk1/Library/CloudStorage/OneDrive-unige.ch/icu_research/prehospital/analgesia/data/Liste Notärzte-1.xlsx'

In [ ]:
restrict_to_trauma = True

In [ ]:
# Load and preprocess data
data_df = pd.read_excel(data_path)
medic_df = pd.read_excel(medic_data_path)

# Merge with physician data
medic_df = medic_df.drop_duplicates('Mitglieder mit Einsatzfunktion')
medic_df.rename(columns={'Sex m/w': 'physician_sex'}, inplace=True)
data_df = data_df.merge(medic_df, how='left', left_on='Mitglieder mit Einsatzfunktion', right_on='Mitglieder mit Einsatzfunktion')

# Remove duplicates
data_df = data_df.drop_duplicates(subset=["SNZ Ereignis Nr. "])

# Filter patients with VAS > 3 at scene
data_df = data_df[data_df["VAS_on_scene"] > 3]

print(f"Total patients after filtering: {len(data_df)}")
print(f"Adult patients: {(data_df['Alter '] >= 16).sum()}")
print(f"Pediatric patients: {(data_df['Alter '] < 16).sum()}")

In [ ]:
if restrict_to_trauma:
    n_non_trauma = data_df[data_df['Einteilung (reduziert)'] != 'Unfall'].shape[0]
    print(f'Excluded {n_non_trauma} non-trauma patients')

    # adult non-trauma patients
    n_adult_non_trauma = data_df[(data_df['Einteilung (reduziert)'] != 'Unfall') & (data_df["Alter "] >= 16)].shape[0]
    print(f'Excluded {n_adult_non_trauma} adult non-trauma patients')
    # pediatric non-trauma patients
    n_pediatric_non_trauma = data_df[(data_df['Einteilung (reduziert)'] != 'Unfall') & (data_df["Alter "] < 16)].shape[0]
    print(f'Excluded {n_pediatric_non_trauma} pediatric non-trauma patients')

    data_df = data_df[data_df['Einteilung (reduziert)'] == 'Unfall']

In [ ]:
# Univariate logistic regression analysis
def univariate_logistic_regression(df, outcome_var, predictor_vars):
    """
    Perform univariate logistic regression for each predictor variable
    """
    results = []
    
    for var in predictor_vars:
        # Prepare data
        X = df[[var]].copy()
        y = df[outcome_var]
        
        # Add constant for intercept
        X_with_const = sm.add_constant(X)
        
        # Fit logistic regression
        try:
            model = sm.Logit(y, X_with_const).fit(disp=0)
            
            # Extract results
            coef = model.params[var]
            or_value = np.exp(coef)
            ci_lower = np.exp(model.conf_int().loc[var, 0])
            ci_upper = np.exp(model.conf_int().loc[var, 1])
            p_value = model.pvalues[var]
            
            results.append({
                'Variable': var,
                'Coefficient': coef,
                'OR': or_value,
                'CI_lower': ci_lower,
                'CI_upper': ci_upper,
                'P_value': p_value,
                'OR_CI': f"{or_value:.2f} ({ci_lower:.2f}-{ci_upper:.2f})",
                'P_formatted': f"{p_value:.3f}" if p_value >= 0.001 else "<0.001"
            })
            
        except Exception as e:
            print(f"Error with variable {var}: {e}")
            
    return pd.DataFrame(results)

In [ ]:
# Multivariate logistic regression analysis
def multivariate_logistic_regression(df, outcome_var, predictor_vars):
    """
    Perform multivariate logistic regression
    """
    # Prepare data
    X = df[predictor_vars].copy()
    y = df[outcome_var]
    
    # Add constant for intercept
    X_with_const = sm.add_constant(X)
    
    # Fit logistic regression
    model = sm.Logit(y, X_with_const).fit(disp=0)
    
    # Extract results
    results = []
    for var in predictor_vars:
        coef = model.params[var]
        or_value = np.exp(coef)
        ci_lower = np.exp(model.conf_int().loc[var, 0])
        ci_upper = np.exp(model.conf_int().loc[var, 1])
        p_value = model.pvalues[var]
        
        results.append({
            'Variable': var,
            'Coefficient': coef,
            'OR': or_value,
            'CI_lower': ci_lower,
            'CI_upper': ci_upper,
            'P_value': p_value,
            'OR_CI': f"{or_value:.2f} ({ci_lower:.2f}-{ci_upper:.2f})",
            'P_formatted': f"{p_value:.3f}" if p_value >= 0.001 else "<0.001"
        })
    
    results_df = pd.DataFrame(results)
    
    # Add model statistics
    model_stats = {
        'Log-likelihood': model.llf,
        'AIC': model.aic,
        'BIC': model.bic,
        'Pseudo R-squared': model.prsquared,
        'LLR p-value': model.llr_pvalue
    }
    
    return results_df, model_stats, model

In [ ]:
def preprocess_body_region(df):
    # fill na in Körperregion with Körperregion2
    df['Körperregion'] = df['Körperregion'].fillna(df['Körperregion2'])
    # set to lower
    df['Körperregion'] = df['Körperregion'].str.lower()
    # replace kopf and 'schädel-hirn/hals' and 'schädel-hirn/gesicht' and 'schädel-hirn/hals' with schädel-hirn
    df['Körperregion'] = df['Körperregion'].str.replace('kopf', 'schädel-hirn')
    df['Körperregion'] = df['Körperregion'].str.replace('schädel-hirn/hals', 'schädel-hirn')
    df['Körperregion'] = df['Körperregion'].str.replace('schädel-hirn/gesicht', 'schädel-hirn')
    df['Körperregion'] = df['Körperregion'].str.replace('schädel-hirn/hals', 'schädel-hirn')
    # replace 'kopf/gesicht' / 'augen' / 'kopf/hals' / 'kopf (gehör)' / 'kopf/gesicht/hals'  and 'schädel-hirn (gehör)' with 'gesicht'
    # df['Körperregion'] = df['Körperregion'].str.replace({'kopf/gesicht': 'gesicht', 'augen': 'gesicht', 'kopf/hals': 'gesicht', 'kopf (gehör)': 'gesicht', 'kopf/gesicht/hals': 'gesicht'})
    df['Körperregion'] = df['Körperregion'].str.replace('kopf/gesicht', 'gesicht')
    df['Körperregion'] = df['Körperregion'].str.replace('kopf/hals', 'gesicht')
    df['Körperregion'] = df['Körperregion'].str.replace('kopf (gehör)', 'gesicht')
    df['Körperregion'] = df['Körperregion'].str.replace('kopf/gesicht/hals', 'gesicht')
    df['Körperregion'] = df['Körperregion'].str.replace('augen', 'gesicht')
    df['Körperregion'] = df['Körperregion'].str.replace('schädel-hirn (gehör)', 'gesicht')
    # replace 'rücken' with 'wirbelsäule'
    df['Körperregion'] = df['Körperregion'].str.replace('rücken', 'wirbelsäule')
    # replace 'rump' with 'thorax'
    df['Körperregion'] = df['Körperregion'].str.replace('rumpf', 'thorax')
    # replace  'obere extremiät' and 'obere extermität' with 'obere extremität'    
    df['Körperregion'] = df['Körperregion'].str.replace('obere extremiät', 'obere extremität')
    df['Körperregion'] = df['Körperregion'].str.replace('obere extermität', 'obere extremität')
    # replace 'untere extermität' with 'untere extremität'    
    df['Körperregion'] = df['Körperregion'].str.replace('untere extermität', 'untere extremität')
    # replace '' with pd.NA
    df['Körperregion'] = df['Körperregion'].replace('', pd.NA)

    return df

# Adult Patients (≥16 years)

In [ ]:
# Prepare adult-only dataset (≥16 years)
adult_df = data_df[data_df['Alter '] >= 16].copy()

# insufficient pain management (VAS on arrival > 3)
adult_df['insufficient_pain_mgmt'] = (adult_df['VAS_on_arrival'] > 3).astype(int)

# Create predictor variables for adult patients (exclude pediatric variable)
adult_df['age'] = adult_df['Alter ']
adult_df['male_patient'] = (adult_df['Geschlecht'] == 'Männlich').astype(int)
adult_df['male_physician'] = (adult_df['physician_sex'] == 'm').astype(int)
adult_df['primary_mission'] = (adult_df['Einsatzart'] == 'Primär').astype(int)
adult_df['night_mission'] = (adult_df['Tag oder Nacht'] == 'Nacht').astype(int)
# separate into winter half-year (October to March) and summer half-year (April to September)
adult_df['winter_season'] = np.where(adult_df['Monat'].isin(['Oktober', 'November', 'Dezember', 'Januar', 'Februar', 'März']), 1, 0).astype(int)
adult_df['trauma'] = adult_df['Einteilung (reduziert)'].str.contains('Unfall', na=False).astype(int)
adult_df['winch_extraction'] = adult_df['Bergungen'].str.contains('Winde', na=False).astype(int)
adult_df['vas_scene'] = adult_df['VAS_on_scene']
adult_df['mission_duration'] = (
    pd.to_datetime(adult_df['Übergabezeit'], format='%d.%m.%Y %H:%M:%S') - 
    pd.to_datetime(adult_df['Erstbefund'], format='%d.%m.%Y %H:%M:%S')
).dt.total_seconds() / 60

adult_df = preprocess_body_region(adult_df)

# number of regions involved per patient
adult_df['n_regions_involved'] = adult_df['Körperregion'].str.count(',') + 1
# if Körperregion is polytrauma, set n_regions_involved to 2
adult_df.loc[adult_df['Körperregion'].str.contains('polytrauma', na=False), 'n_regions_involved'] = 2

# regional involvement
adult_df['extremities_involved'] = adult_df['Körperregion'].str.contains('obere extremität|untere extremität', na=False).astype(int)
adult_df['thorax_involved'] = adult_df['Körperregion'].str.contains('thorax', na=False).astype(int)
# abdomen or pelvis involvement
adult_df['abdomen_pelvis_involved'] = adult_df['Körperregion'].str.contains('abdomen|becken', na=False).astype(int)
adult_df['spine_involved'] = adult_df['Körperregion'].str.contains('wirbelsäule', na=False).astype(int)
# face/neck involvement
adult_df['face_neck_involved'] = adult_df['Körperregion'].str.contains('gesicht|hals', na=False).astype(int)
# tbi 
adult_df['tbi_involved'] = adult_df['Körperregion'].str.contains('schädel-hirn', na=False).astype(int)
# polytrauma
adult_df['suspected_polytrauma'] = adult_df['Körperregion'].str.contains('polytrauma', na=False).astype(int)

# decompose patient / doctor sex into combinations
# male / male
adult_df['dr_male_pt_male'] = adult_df['male_physician'] & adult_df['male_patient']
adult_df['dr_female_pt_female'] = ((adult_df['male_physician'] == 0) & (adult_df['male_patient'] == 0)).astype(int)
adult_df['dr_male_pt_female'] = (adult_df['male_physician'] & (adult_df['male_patient'] == 0)).astype(int)
adult_df['dr_female_pt_male'] = ((adult_df['male_physician'] == 0) & adult_df['male_patient']).astype(int)

# Create the same additional variables for adult patients
adult_df['short_scene_time'] = (adult_df['mission_duration'] < 30).astype(int)

# Create medication dose variables (matching Table 1 approach)
adult_df['fentanyl_dose'] = 0
adult_df['ketamine_dose'] = 0
adult_df['esketamine_dose'] = 0
adult_df['morphine_dose'] = 0
adult_df['Alle Medikamente'] = adult_df['Alle Medikamente'].str.replace(',', ';')  # replace commas with semicolons for consistency
for i, row in adult_df.iterrows():
    if pd.isna(row['Alle Medikamente']) or row['Alle Medikamente'] == 0:
        continue
    for analgetic in row['Alle Medikamente'].split(';'):
        if analgetic.strip() == '':
            continue
        # remove mcg or mg from dose
        if '7IE' in analgetic:
                print(f"Skipping dose with 7IE: {analgetic}")
                continue

        analgetic = analgetic.replace('mcg', '').replace('mg', '').strip()
        if 'Fentanyl' in analgetic and '/h' not in analgetic:
            dose = analgetic.split('Fentanyl')[-1].strip()
            adult_df.at[i, 'fentanyl_dose'] += float(dose) 
        elif 'Fentanyl' in analgetic and '/h' in analgetic:
            dose = analgetic.split('Fentanyl')[-1].strip().replace('/h', '')
            dose = float(dose) * adult_df.at[i, 'mission_duration']  
            adult_df.at[i, 'fentanyl_dose'] += float(dose)
        elif 'Ketamin' in analgetic or 'Ketamine' in analgetic:
            dose = analgetic.split('Ketamin')[-1].strip()
            adult_df.at[i, 'ketamine_dose'] += float(dose)
        elif 'Esketamin' in analgetic:
            dose = analgetic.split('Esketamin')[-1].strip()
            adult_df.at[i, 'esketamine_dose'] += float(dose)
        elif 'Morphin' in analgetic or 'Morphine' in analgetic:
            dose = analgetic.split('Morphin')[-1].strip()
            adult_df.at[i, 'morphine_dose'] += float(dose)

# fentanyl given
adult_df['fentanyl_given'] = adult_df['fentanyl_dose'] > 0
adult_df['morphine_given'] = adult_df['morphine_dose'] > 0
adult_df['ketamine_given'] = adult_df['ketamine_dose'] > 0
adult_df['esketamine_given'] = adult_df['esketamine_dose'] > 0

# Create combined medication variables
adult_df['any_opiate_dose'] = adult_df['morphine_dose'] + adult_df['fentanyl_dose']
adult_df['any_opiate_given'] = (adult_df['morphine_dose'] > 0) | (adult_df['fentanyl_dose'] > 0)
adult_df['any_ketamine_given'] = (adult_df['ketamine_dose'] > 0) | (adult_df['esketamine_dose'] > 0)

# combination therapy (opiate + ketamine) vs monotherapy
adult_df['opiate_ketamine_combination_therapy'] = (adult_df['any_opiate_given'] & adult_df['any_ketamine_given']).astype(int)

# Create no_analgesic variable: patients who receive no analgesic
# (no_analgesic corresponds to a dose of any_opiate and any_ketamine of 0)
adult_df['no_analgesic'] = ((adult_df['any_opiate_given'] == 0) & (adult_df['any_ketamine_given'] == 0)).astype(int)

# create outcome variable: insufficient pain management (VAS on arrival > 3) and no analgesic given
adult_df['persisting_untreated_pain'] = ((adult_df['insufficient_pain_mgmt'] == 1) & (adult_df['no_analgesic'] == 1)).astype(int)

# Variables for adult model (including the 3 required variables)
adult_model_vars = ['persisting_untreated_pain', 'age', 'NACA',
                    'dr_male_pt_male', 'dr_female_pt_female', 'dr_male_pt_female', 'dr_female_pt_male',
                    'n_regions_involved', 'suspected_polytrauma', 'extremities_involved', 'thorax_involved', 'abdomen_pelvis_involved',
                    'spine_involved', 'face_neck_involved', 'tbi_involved',
                   'mission_duration', 'primary_mission', 'night_mission',
                    'winch_extraction', 'vas_scene', 'winter_season']

# print number of na per variable in adult_model_vars
for var in adult_model_vars:
    n_na = adult_df[var].isna().sum()
    print(f"{var}: {n_na} NA values")

adult_df_clean = adult_df[adult_model_vars].dropna()

print(f"Adult patients included in model: {len(adult_df_clean)}")

In [ ]:
# Adult predictor variables (including the 3 required variables)
adult_predictor_vars = ['age', 
                        'dr_male_pt_male',  'dr_male_pt_female', 'dr_female_pt_male',
                          'n_regions_involved', 'suspected_polytrauma', 'extremities_involved', 'thorax_involved', 'abdomen_pelvis_involved',
                    'spine_involved', 'face_neck_involved', 'tbi_involved',
                        'mission_duration', 'NACA',
                        'primary_mission', 'night_mission', 'winter_season',
                       'winch_extraction', 'vas_scene']

# Univariate analysis for adults
adult_univariate_results = univariate_logistic_regression(adult_df_clean, 'persisting_untreated_pain', adult_predictor_vars)

print("ADULT PATIENTS - Univariate Analysis:")
print("=" * 60)
for _, row in adult_univariate_results.iterrows():
    print(f"{row['Variable']}: OR = {row['OR_CI']}, p = {row['P_formatted']}")

# Multivariate analysis for adults (including the 3 required variables)
adult_multivariate_vars = adult_predictor_vars

adult_multivariate_results, adult_model_stats, adult_fitted_model = multivariate_logistic_regression(
    adult_df_clean, 'persisting_untreated_pain', adult_multivariate_vars)

print("\nPersisting untreated pain: (VAS on arrival > 3 and no analgesic given)")
print(f"\nADULT PATIENTS - Multivariate Analysis:")
print("=" * 60)
for _, row in adult_multivariate_results.iterrows():
    print(f"{row['Variable']}: OR = {row['OR_CI']}, p = {row['P_formatted']}")

print(f"\nAdult Model Statistics:")
print(f"Pseudo R-squared: {adult_model_stats['Pseudo R-squared']:.3f}")
print(f"AIC: {adult_model_stats['AIC']:.2f}")
print(f"LLR p-value: {adult_model_stats['LLR p-value']:.3f}")

adult_multivariate_results

In [ ]:
from statsmodels.stats.contrast import ContrastResults

# 1. Global Wald test (all dr/patient sex variables jointly)
sex_vars = ['dr_male_pt_male', 'dr_male_pt_female', 'dr_female_pt_male']

# Build restriction matrix for joint hypothesis H0: all sex_vars = 0
R = np.zeros((len(sex_vars), len(adult_fitted_model.params)))
for i, var in enumerate(sex_vars):
    idx = adult_fitted_model.model.exog_names.index(var)
    R[i, idx] = 1

global_test = adult_fitted_model.wald_test(R)

print("\nGlobal Wald test for doctor/patient sex categories:")
print("=" * 60)
print(global_test)   # Shows chi2 statistic, df, p-value

# Define the variables representing doctor/patient sex categories
sex_vars = ['dr_male_pt_male', 'dr_male_pt_female', 'dr_female_pt_male']

# Perform post-hoc contrasts
# (reference group is implicitly "dr_female_pt_female" since it's omitted)
contrast_results = {}

# Example contrasts:
# 1. dr_male_pt_male vs dr_male_pt_female
contrast_matrix = np.zeros(len(adult_fitted_model.params))
contrast_matrix[adult_fitted_model.model.exog_names.index('dr_male_pt_male')] = 1
contrast_matrix[adult_fitted_model.model.exog_names.index('dr_male_pt_female')] = -1
contrast_results['dr_male_pt_male vs dr_male_pt_female'] = adult_fitted_model.t_test(contrast_matrix)

# 2. dr_male_pt_male vs dr_female_pt_male
contrast_matrix = np.zeros(len(adult_fitted_model.params))
contrast_matrix[adult_fitted_model.model.exog_names.index('dr_male_pt_male')] = 1
contrast_matrix[adult_fitted_model.model.exog_names.index('dr_female_pt_male')] = -1
contrast_results['dr_male_pt_male vs dr_female_pt_male'] = adult_fitted_model.t_test(contrast_matrix)

# 3. dr_male_pt_female vs dr_female_pt_male
contrast_matrix = np.zeros(len(adult_fitted_model.params))
contrast_matrix[adult_fitted_model.model.exog_names.index('dr_male_pt_female')] = 1
contrast_matrix[adult_fitted_model.model.exog_names.index('dr_female_pt_male')] = -1
contrast_results['dr_male_pt_female vs dr_female_pt_male'] = adult_fitted_model.t_test(contrast_matrix)

# dr_male_pt_female vs dr_female_pt_female (reference group)
contrast_matrix = np.zeros(len(adult_fitted_model.params))
contrast_matrix[adult_fitted_model.model.exog_names.index('dr_male_pt_female')] = 1
contrast_results['dr_male_pt_female vs dr_female_pt_female'] = adult_fitted_model.t_test(contrast_matrix)


print("\nPairwise post-hoc contrasts for doctor/patient sex categories:")
print("=" * 60)

contrast_summary = []

for name, res in contrast_results.items():
    est = res.effect.item()    # log-odds difference
    se = res.sd.item()
    pval = res.pvalue.item()
    
    or_val = np.exp(est)
    ci_low = np.exp(est - 1.96 * se)
    ci_high = np.exp(est + 1.96 * se)
    
    print(f"{name}: OR = {or_val:.2f} ({ci_low:.2f}-{ci_high:.2f}), p = {pval:.4f}")
    
    contrast_summary.append({
        "Contrast": name,
        "Logit_diff": est,
        "OR": or_val,
        "CI_lower": ci_low,
        "CI_upper": ci_high,
        "p_value": pval
    })

# Optional: save results in a DataFrame
contrast_df = pd.DataFrame(contrast_summary)

In [ ]:
# Create formatted Table 3: Factors for Persisting Untreated Pain
def create_table3(univariate_df, multivariate_df):
    """
    Create a publication-ready Table 3
    """
    # Define variable labels for better presentation
    variable_labels = {
        'age': 'Age (years)',
        'male_patient': 'Male patient',
        'male_physician': 'Male physician',
        'short_scene_time': 'Short time at scene (<30 min)',
        'no_analgesic': 'No analgesic given',
        'pediatric': 'Pediatric patient (<16 years)',
        'primary_mission': 'Primary mission',
        'night_mission': 'Night mission',
        'trauma': 'Trauma',
        'winch_extraction': 'Winch extraction',
        'vas_scene': 'VAS at scene',
        'mission_duration': 'Mission duration (minutes)',
        'dr_male_pt_male': 'Male doctor - Male patient',
        'dr_male_pt_female': 'Male doctor - Female patient', 
        'dr_female_pt_male': 'Female doctor - Male patient',
        'n_regions_involved': 'Number of regions involved',
        'suspected_polytrauma': 'Suspected polytrauma',
        'extremities_involved': 'Extremities involved',
        'thorax_involved': 'Thorax involved',
        'abdomen_pelvis_involved': 'Abdomen/pelvis involved',
        'spine_involved': 'Spine involved',
        'face_neck_involved': 'Face/neck involved',
        'tbi_involved': 'Traumatic brain injury',
        'NACA': 'NACA score',
        'winter_season': 'Winter season'
    }
    
    # Create combined table
    table3_data = []
    
    # Merge univariate and multivariate results
    for var in univariate_df['Variable']:
        uni_row = univariate_df[univariate_df['Variable'] == var].iloc[0]
        
        # Check if variable is in multivariate model
        multi_row = multivariate_df[multivariate_df['Variable'] == var]
        
        if len(multi_row) > 0:
            multi_row = multi_row.iloc[0]
            multi_or_ci = multi_row['OR_CI']
            multi_p = multi_row['P_formatted']
        else:
            multi_or_ci = '-'
            multi_p = '-'
        
        table3_data.append({
            'Variable': variable_labels.get(var, var),
            'Univariate_OR_CI': uni_row['OR_CI'],
            'Univariate_P': uni_row['P_formatted'],
            'Multivariate_OR_CI': multi_or_ci,
            'Multivariate_P': multi_p
        })
    
    table3_df = pd.DataFrame(table3_data)
    return table3_df

In [ ]:
# Create Table 3: Adult Patients
adult_table3 = create_table3(adult_univariate_results, adult_multivariate_results)

print("Table 3. Factors for Persisting Untreated Pain - ADULT PATIENTS (≥16 years)")
print("=" * 80)
print(f"{'Variable':<30} {'Univariate':<20} {'P-value':<10} {'Multivariate':<20} {'P-value':<10}")
print(f"{'':30} {'OR (95% CI)':<20} {'':10} {'OR (95% CI)':<20} {'':10}")
print("-" * 80)

for _, row in adult_table3.iterrows():
    print(f"{row['Variable']:<30} {row['Univariate_OR_CI']:<20} {row['Univariate_P']:<10} {row['Multivariate_OR_CI']:<20} {row['Multivariate_P']:<10}")

print("\nOR = Odds Ratio, CI = Confidence Interval")
print(f"Model includes {len(adult_df_clean)} adult patients")
print(f"Persisting untreated pain: {adult_df_clean['persisting_untreated_pain'].sum()}/{len(adult_df_clean)} ({adult_df_clean['persisting_untreated_pain'].mean():.1%})")

# Show significant variables for adults
adult_significant_vars = adult_multivariate_results[adult_multivariate_results['P_value'] < 0.05]
if len(adult_significant_vars) > 0:
    print(f"\nSignificant Variables in Adult Multivariate Model (p < 0.05):")
    for _, row in adult_significant_vars.iterrows():
        direction = "increased" if row['OR'] > 1 else "decreased"
        print(f"- {row['Variable']}: OR = {row['OR']:.2f}, {direction} odds of persisting untreated pain")
else:
    print("\nNo variables reached statistical significance (p < 0.05) in adult model")

adult_table3

In [ ]:
# save adult_table3 to csv
# adult_table3.to_csv('/Users/jk1/Library/CloudStorage/OneDrive-unige.ch/icu_research/prehospital/analgesia/analysis/adult_trauma/table3.csv', index=False)

# Summary Figures

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Figure 3: Forest Plot - 
def create_adult_forest_plot():
    """
    Create a forest plot showing odds ratios for adult population only
    """
    fig, ax = plt.subplots(figsize=(12, 10))
    
    # Variable labels for better presentation
    var_labels = {
        'age': 'Age (years)',
        'dr_male_pt_male': 'Male doctor - Male patient',
        'dr_male_pt_female': 'Male doctor - Female patient', 
        'dr_female_pt_male': 'Female doctor - Male patient',
        'n_regions_involved': 'Number of regions involved',
        'suspected_polytrauma': 'Suspected polytrauma',
        'extremities_involved': 'Extremities involved',
        'thorax_involved': 'Thorax involved',
        'abdomen_pelvis_involved': 'Abdomen/pelvis involved',
        'spine_involved': 'Spine involved',
        'face_neck_involved': 'Face/neck involved',
        'tbi_involved': 'Traumatic brain injury',
        'mission_duration': 'Mission duration (minutes)',
        'NACA': 'NACA score',
        'primary_mission': 'Primary mission',
        'night_mission': 'Night mission',
        'winter_season': 'Winter season',
        'winch_extraction': 'Winch extraction',
        'vas_scene': 'VAS at scene'
    }
    
    if len(adult_multivariate_results) > 0:
        # Prepare data from adult multivariate results
        variables = []
        ors = []
        ci_lower = []
        ci_upper = []
        pvals = []
        
        # Sort by OR magnitude for better visualization
        adult_results_sorted = adult_multivariate_results.sort_values('OR', ascending=True)
        
        for _, row in adult_results_sorted.iterrows():
            var = row['Variable']
            variables.append(var_labels.get(var, var))
            ors.append(row['OR'])
            ci_lower.append(row['CI_lower'])
            ci_upper.append(row['CI_upper'])
            pvals.append(row['P_value'])
        
        y_positions = np.arange(len(variables))
        
        # Color points based on significance
        colors = ['#FF4444' if p < 0.05 else '#666666' for p in pvals]
        marker_sizes = [120 if p < 0.05 else 80 for p in pvals]
        
        # Plot the forest plot
        for i, (or_val, ci_l, ci_u, p_val) in enumerate(zip(ors, ci_lower, ci_upper, pvals)):
            color = '#FF4444' if p_val < 0.05 else '#666666'
            size = 120 if p_val < 0.05 else 80
            
            # Plot point estimate
            ax.scatter(or_val, y_positions[i], color=color, s=size, zorder=3, alpha=0.8)
            
            # Plot confidence interval
            ax.plot([ci_l, ci_u], [y_positions[i], y_positions[i]], 
                   color=color, linewidth=2, alpha=0.7, zorder=2)
            
            # Plot CI caps
            cap_size = 0.1
            ax.plot([ci_l, ci_l], [y_positions[i]-cap_size, y_positions[i]+cap_size], 
                   color=color, linewidth=2, alpha=0.7, zorder=2)
            ax.plot([ci_u, ci_u], [y_positions[i]-cap_size, y_positions[i]+cap_size], 
                   color=color, linewidth=2, alpha=0.7, zorder=2)
            
            # Add OR and CI text
            # ax.text(max(ors) * 1.3, y_positions[i], 
            ax.text(5.5, y_positions[i], 
                   f'{or_val:.2f} ({ci_l:.2f}-{ci_u:.2f})', 
                   va='center', fontsize=9, fontweight='bold' if p_val < 0.05 else 'normal')
        
        # Reference line at OR = 1
        ax.axvline(x=1, color='black', linestyle='--', linewidth=1, alpha=0.8)
        
        # Formatting
        ax.set_xlabel('Odds Ratio (95% CI)', fontsize=14, fontweight='bold')
        ax.set_yticks(y_positions)
        ax.set_yticklabels(variables, fontsize=11)
        
        # Set x-axis limits with some padding
        x_min = min(min(ci_lower) * 0.8, 0.1)
        x_max = max(max(ci_upper), max(ors)) * 1.3
        ax.set_xlim(x_min, x_max)
        
        # Add grid
        ax.grid(True, alpha=0.3, axis='x')
        
        # Title and subtitle
        ax.set_title('Factors Associated with persisting untreated pain', 
                    fontsize=16, fontweight='bold', pad=20)
        
        # Add legend
        from matplotlib.lines import Line2D
        legend_elements = [
            Line2D([0], [0], marker='o', color='w', markerfacecolor='#FF4444', markersize=10, 
                   label=f'Significant (p < 0.05) [n={sum(1 for p in pvals if p < 0.05)}]'),
            Line2D([0], [0], marker='o', color='w', markerfacecolor='#666666', markersize=8, 
                   label=f'Not significant [n={sum(1 for p in pvals if p >= 0.05)}]')
        ]
        # ax.legend(handles=legend_elements, bbox_to_anchor=(1.05, 0), loc='lower left', fontsize=10)
        
        # Add model info text
        model_info = f"""
        Model: n = {len(adult_df_clean)} adult patients
        Outcome: Persisting untreated pain (VAS arrival > 3 & no analgesia)
        """
        # ax.text(1.02, 0.98, model_info.strip(), transform=ax.transAxes, 
        #        fontsize=9, verticalalignment='top', 
        #        bbox=dict(boxstyle="round,pad=0.3", facecolor="lightblue", alpha=0.7))
        
        plt.tight_layout()
        plt.show()
        
        # Print summary
        print("\\nForest Plot Summary - Adult Population:")
        print("=" * 50)
        print(f"Total variables analyzed: {len(variables)}")
        print(f"Significant associations (p < 0.05): {sum(1 for p in pvals if p < 0.05)}")
        print(f"Sample size: {len(adult_df_clean)} adult patients")
        print(f"Outcome rate: {adult_df_clean['persisting_untreated_pain'].mean():.1%}")
        
        # List significant variables
        sig_vars = [(var, or_val, p_val) for var, or_val, p_val in zip(variables, ors, pvals) if p_val < 0.05]
        if sig_vars:
            print("\\nSignificant associations:")
            for var, or_val, p_val in sig_vars:
                direction = "increased" if or_val > 1 else "decreased"
                print(f"  • {var}: OR = {or_val:.2f} ({direction} odds, p = {p_val:.3f})")
        else:
            print("\\nNo variables reached statistical significance (p < 0.05)")
    else:
        print("No adult multivariate results available for forest plot")

    return fig

# Create the adult-only forest plot
fig = create_adult_forest_plot()

In [ ]:
# save
# fig.savefig('/Users/jk1/Library/CloudStorage/OneDrive-unige.ch/icu_research/prehospital/analgesia/analysis/adult_trauma/untreated_persisting_pain_factors.png', dpi=300)